In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

# Load data
data = pd.read_csv('sma_data.csv')

# Preprocess data
data['datetime'] = pd.to_datetime(data['datetime'])
data['sma'] = (data['sma'] - data['sma'].mean()) / data['sma'].std()

# Extract features
window_size = 60  # 1 hour
features = []
for i in range(len(data) - window_size):
    window = data.iloc[i:i+window_size]
    mean_sma = window['sma'].mean()
    std_sma = window['sma'].std()
    slope_sma = (window['sma'].iloc[-1] - window['sma'].iloc[0]) / window_size
    features.append([mean_sma, std_sma, slope_sma])

# Create labels
labels = np.zeros(len(features))
for i in range(len(labels)):
    if data.iloc[i+window_size]['sma'] > data.iloc[i]['sma'] + 0.1:
        labels[i] = 1  # maneuver detected

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Train random forest model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Evaluate model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.3f}')

# Plot detected maneuvers
detected_maneuvers = np.where(y_pred == 1)[0]
plt.plot(data['datetime'], data['sma'])
plt.scatter(data['datetime'].iloc[detected_maneuvers], data['sma'].iloc[detected_maneuvers], c='r')
plt.show()